![GPT](images/gpt-arch-1.png)

In [ ]:
import torch
import torch.nn as nn
from modules import TransformerBlock, LayerNorm

class GPTModel(nn.Module):

    def __init__(self, config):
        super().__init__()
        self.tok_emb = nn.Embedding(config["vocab_size"], config["emb_dim"])
        self.pos_emb = nn.Embedding(config["context_length"], config["emb_dim"])
        self.drop_emb = nn.Dropout(config["drop_rate"])
        self.transformerlayers = nn.Sequential(*[TransformerBlock(config) for _ in range(config["number_of_layers"])])
        
        self.finalnorm = LayerNorm(config["emb_dim"])
        self.out_head = nn.Linear(config["emb_dim"], config["vocab_size"], bias=False)

    def forward(self, input_idx):
        batch_size, seq_len = input_idx.shape
        token_embeddings = self.tok_emb(input_idx)
        positional_embeddings = self.pos_emb(torch.arange(seq_len, device=input_idx.device))
        x = token_embeddings + positional_embeddings
        x = self.drop_emb(x)
        x = self.transformerlayers(x)
        x = self.finalnorm(x)
        logits = self.out_head(x)
        return logits


Weight Tying 

a concept to save overall memory footprint and computation, used in original GPT2 architecture, where the token embedding weights are used in its output layer, otherwise it would've been 163M params instead of 124M.

however, modern llm architectures use separate weights at these places as it gives better model training and performance.

In [ ]:
model = GPTModel(GPT_CONFIG)

# Example input (batch_size=2, seq_len=10)
input_ids = torch.randint(0, GPT_CONFIG["vocab_size"], (2, 10))
print(input_ids)

with torch.no_grad():
    logits = model(input_ids)

print(f"Input shape: {input_ids.shape}")
print(f"Output shape: {logits.shape}")
print(f"\nModel has {sum(p.numel() for p in model.parameters()):,} parameters")


tensor([[48456, 25800, 42114, 10130, 24711, 47351,  3781, 36012, 16549, 23321],
        [42968, 46211, 23438, 44343,  3518,  9778, 20626, 32497, 42517, 33764]])
Input shape: torch.Size([2, 10])
Output shape: torch.Size([2, 10, 50257])

Model has 163,009,536 parameters


**Generating Text**

![](images/gpt-arch-2.png)

We will implement Greedy Decoding, which is the most straightforward way an LLM generates text. It predicts one token at a time by always choosing the single most likely next word.

There are more algorithms of generating text that brings in creativity by upating the softmax function

In [4]:
from modules import generate_text_simple
# Testing, but i need tokenizer for it
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

start_context = "Hello, I am"
encoded = tokenizer.encode(start_context)
print("encoded:", encoded)

encoded_tensor = torch.tensor(encoded).unsqueeze(0)
print("encoded_tensor.shape:", encoded_tensor.shape)

model.eval() # disable dropout

out = generate_text_simple(
    model=model,
    idx=encoded_tensor, 
    max_new_tokens=6, 
    context_size=GPT_CONFIG["context_length"]
)

print("Output:", out)
print("Output length:", len(out[0]))

decoded_text = tokenizer.decode(out.squeeze(0).tolist())
print(decoded_text)

encoded: [15496, 11, 314, 716]
encoded_tensor.shape: torch.Size([1, 4])
Output: tensor([[15496,    11,   314,   716, 45495, 21824, 25484, 28517, 24313, 25730]])
Output length: 10
Hello, I am� grains Invasion harmedodo Sherlock
